In [ ]:
!pip install ultralytics supervision roboflow
!pip install -U ipywidgets
!pip install -U albumentations
import os

In [ ]:
import cv2
import supervision as sv
import random
from pathlib import Path
import glob
import os
from IPython.display import Image
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import ultralytics
ultralytics.checks()

Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Setup complete ✅ (12 CPUs, 53.0 GB RAM, 33.3/235.7 GB disk)


In [ ]:
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/vehicle_dataset_split/daytime/weights/bestYOLOv11x45_100e_lr=0.0015.pt")

Ultralytics 8.3.28 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLO11x summary (fused): 464 layers, 56,831,644 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /content/drive/MyDrive/vehicle_dataset_split/daytime/val/labels.cache... 1379 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1379/1379 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/drive/MyDrive/vehicle_dataset_split/daytime/val/images/cam_01_00961.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/drive/MyDrive/vehicle_dataset_split/daytime/val/images/cam_04_00346.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/drive/MyDrive/vehicle_dataset_split/daytime/val/images/cam_04_00915.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/drive/MyDrive/vehicle_dataset_split/daytime/val/images/cam_10_00136.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 87/87 [00:59<00:00,  1.46it/s]


                   all       1379      20033      0.903      0.916      0.956      0.687
             motorbike       1286      13288      0.911      0.901      0.958       0.61
                   car       1057       3571      0.907      0.926      0.959       0.69
                   bus        551        803      0.902      0.915      0.953      0.737
             container        883       2371      0.893      0.923      0.953      0.711
Speed: 0.2ms preprocess, 14.7ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b2c012911e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

In [ ]:
test_img_path = list(Path("/content/drive/MyDrive/vehice_dataset/test/images").glob("*.jpg")) # test data
test_img_path.sort()
len(test_img_path)

2060

In [ ]:
for i in range(50): # inference code
  index = random.randrange(len(test_img_path))
  image = cv2.imread(test_img_path[index])
  results = mock_model(image)[0]
  detections = sv.Detections.from_ultralytics(results)

  bounding_box_annotator = sv.BoundingBoxAnnotator()
  label_annotator = sv.LabelAnnotator()

  annotated_image = bounding_box_annotator.annotate(
      scene=image, detections=detections)
  annotated_image = label_annotator.annotate(
      scene=annotated_image, detections=detections)
  fig = plt.figure(figsize=(50, 50))      # đoạn này tôi không tạo matplotlib lắm nên ông cứ tuỳ chỉnh theo ý ông nhé
  fig.add_subplot(10, 5, i + 1)
  plt.imshow(annotated_image, aspect='equal')
  plt.axis(False)

## Writing-prediction-results-to-predict.txt procedure
##### 0.image = cv2.imread(test_img_path)
##### 1.results = model(image)[0]
##### 2.detections = sv.Detections.from_ultralytics(results)
##### 3.filepath = Path("/content/drive/MyDrive/vehice_dataset/predict.txt")
##### 4.bounding_boxes = results.boxes.xywhn.cpu().numpy()
##### 5.predicted_class = detections.class_id
##### 6.conf = detections.confidence
##### 7.insert_prediction(bounding_boxes, predicted_class,conf, filepath)

In [ ]:
model_with_best_weights = YOLO('/content/drive/MyDrive/vehicle_dataset_split/daytime/weights/bestYOLOv11x45_100e_lr=0.0015.pt')

In [ ]:
def insert_prediction(img_path, bounding_boxes, predicted_class,conf, filepath):
  with open(filepath, "a") as f:
    for i in range(len(bounding_boxes)):
      s = ""
      s += img_path.name + " "
      s += str(predicted_class[i]) + " "
      for j in range(4):
        s += str(bounding_boxes[i][j]) + " "
      s += str(conf[i]) + "\n"
      f.write(s)
    f.close()

In [ ]:
# clear predict file
filepath = Path("/content/drive/MyDrive/vehice_dataset/predict.txt")
open(filepath, 'w').close()

In [ ]:
from tqdm.auto import tqdm
for i in tqdm(range(len(test_img_path))):
  image = cv2.imread(test_img_path[i])
  results = model_with_best_weights(image)[0]
  detections = sv.Detections.from_ultralytics(results)
  bounding_boxes = results.boxes.xywhn.cpu().numpy()
  predicted_class = detections.class_id
  conf = detections.confidence
  insert_prediction(test_img_path[i], bounding_boxes, predicted_class,conf, filepath)
# phần dưới là minh hoạ

  0%|          | 0/2060 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
0: 384x640 7 motorbikes, 4 cars, 19.8ms
Speed: 2.0ms preprocess, 19.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 motorbikes, 3 cars, 20.0ms
Speed: 2.2ms preprocess, 20.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 motorbikes, 3 cars, 19.7ms
Speed: 2.0ms preprocess, 19.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 motorbikes, 3 cars, 19.7ms
Speed: 2.1ms preprocess, 19.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 motorbikes, 3 cars, 20.1ms
Speed: 2.3ms preprocess, 20.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 motorbikes, 3 cars, 20.4ms
Speed: 2.2ms preprocess, 20.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 motorbikes, 3 cars, 19.9ms
Speed: 2.0ms preprocess, 19.9ms inference, 1.1ms postprocess per image at shape (